In [ ]:
#Import some libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Import some sci-kit learn libraries to make a simple classifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Data merge

In [ ]:
#Read in the labelled data
dftrain=pd.read_csv("/content/drive/My Drive/Colab Notebooks/aa/data_training.csv",header=0)
#And the target tweet ids for the solution (and the dummy classifcations)
dfsolution=pd.read_csv("/content/drive/My Drive/Colab Notebooks/aa/data_example_random.csv",header=0)
#Hydrate following the steps in the Data page.
#Read in the hydrated data
dfhydrate=pd.read_csv("/content/drive/My Drive/Colab Notebooks/aa/train_data.csv",header=0)
#Read in the hydrated data for the target tweet ids
dftarget=pd.read_csv("/content/drive/My Drive/Colab Notebooks/aa/test_data.csv",header=0)

In [ ]:
#Merge the two datasets (tweet ids from data_training and the tweets from hydrated_training). 
#Note: the number of tweetids availble to hydrate may have changed since the labelled dataset was made.
df=pd.merge(left=dfhydrate,right=dftrain, left_on='id', right_on='tweetid')
df_soltion=pd.merge(left=dftarget,right=dfsolution, left_on='id', right_on='tweetid',how='right')
df

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,retweet_count,reweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_screen_name,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name.1,user_statuses_count,user_time_zone,user_urls,user_verified,tweetid,tweet_label_int
0,NaN,Wed Aug 02 05:51:00 +0000 2017,Research autism vaccines,NaN,NaN,0,892623748146249729,NaN,NaN,NaN,en,NaN,NaN,0,NaN,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",#Research into cause of #autism hasn't occurre...,https://twitter.com/RobinSmithAK/status/892623...,Fri Feb 06 19:09:22 +0000 2015,RobinSmithAK,False,NaN,12007,339,535,71,NaN,Robin Smith,RobinSmithAK,29849,NaN,NaN,False,892623748146249729,0
1,NaN,Thu Feb 01 21:08:36 +0000 2018,NaN,NaN,NaN,2,959171650305404928,SocialPowerOne1,9.591479e+17,1.381089e+09,en,NaN,NaN,3,NaN,NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...","@SocialPowerOne1 “Kenneth Copeland, member of ...",https://twitter.com/raygivens99/status/9591716...,Thu Jan 10 00:37:53 +0000 2013,raygivens99,False,I will decide for myself. I bookmark news & op...,30024,263,1032,21,canada,Fair,raygivens99,53102,NaN,NaN,False,959171650305404928,0
2,NaN,Fri Apr 06 05:12:12 +0000 2018,NaN,NaN,NaN,0,982123785124917248,ProfTimNoakes,9.819910e+17,5.493045e+08,en,NaN,NaN,0,NaN,NaN,"<a href=""http://www.tweetcaster.com"" rel=""nofo...",@ProfTimNoakes stop saving heart attack deaths...,https://twitter.com/swm0904/status/98212378512...,Sat Apr 23 11:44:40 +0000 2011,swm0904,False,Don't believe everything you read. There is su...,304,88,329,7,Brisbane,Smoked,swm0904,9566,NaN,NaN,False,982123785124917248,0
3,NaN,Tue May 15 03:03:06 +0000 2018,POTUS health medicine illness pharmacy drug dr...,NaN,https://twitter.com/HealthRanger/status/996118...,0,996224425753133056,NaN,NaN,NaN,en,NaN,False,0,NaN,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",The first flu vaccine under #POTUS Trump was 9...,https://twitter.com/cureworks/status/996224425...,Sat May 24 20:00:55 +0000 2008,cureworks,False,"Computer Underground Railroad Ent., Founder & ...",30607,5289,6048,533,United States,J. Nayer Hardin,cureworks,83844,NaN,http://www.nayer.blogspot.com,False,996224425753133056,1
4,NaN,Tue Apr 18 12:25:17 +0000 2017,DeliverForGood,NaN,NaN,2,854309857738051584,NaN,NaN,NaN,en,NaN,NaN,0,NaN,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Vaccinating against HPV costs just $10-$25 per...,https://twitter.com/MalathyIyerTOI/status/8543...,Fri Oct 10 11:24:38 +0000 2014,MalathyIyerTOI,False,Senior Editor (Health) at The Times of India,4417,4618,3037,57,Mumbai,Malathy Iyer,MalathyIyerTOI,5264,NaN,NaN,True,854309857738051584,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8367,NaN,Thu Aug 29 04:25:33 +0000 2019,NaN,https://twitter.com/A51FR3D/status/11669298589...,http://dlvr.it/RC4gNC,0,1166929858921779201,NaN,NaN,NaN,en,NaN,False,0,NaN,NaN,"<a href=""https://dlvrit.com/"" rel=""nofollow"">d...",Pinterest to direct vaccine searches to health...,https://twitter.com/A51FR3D/status/11669298589...,Thu Aug 09 22:09:20 +0000 2007,A51FR3D,False,Managing Director @PMA_Accountants in Chingfor...,1497,534,1340,118,"London, UK",Asif Patel,A51FR3D,75143,NaN,http://www.asifpatel.com,False,1166929858921779201,0
8368,NaN,Mon Oct 07 06:31:28 +0000 2019,NaN,NaN,NaN,2,1181094671763869696,CCbucko,1.181094e+18,8.660691e+17,en,NaN,NaN,0,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",@CCbucko The right doesn't know how use vaccin...,https://twitter.com/jlebrech/status/1181094671...,Thu Apr 09 13:46:57 +0000 2009,jlebrech,False,NaN,12638,884,2360,22,NaN,Joseph#,jlebrech,36400,NaN,NaN,False,1181094671763869696,0
8369,N

# Data Cleaning 

In [ ]:
#get all sentences
keeps = [chr(i) for i in range(97,123)] #+ ["0",'1','2','3','4','5','6','7','8','9']
all_sentences=[]
for review in df.text + df_soltion.text:
    if review is np.NaN:
        continue
    all_sentences.extend(list(review.lower()))

In [ ]:
len(all_sentences)

303959

In [ ]:
# generate romove word list
remove_char = []
all_char_list = list(set(all_sentences))
for i in all_char_list:
  if i not in keeps:
    remove_char.append(i)

remove_chars = "".join(remove_char)
print(remove_chars)

"’|£🤛🏻💊❄🍪😇à🌐è👩💛💯▶⃣🐔»🐾💧😎👏🤔5👆é'🩹‼)❤😳⚕➖
🤱👨😺✨🌎😤🚶ú😔🇳🌍⚛+✌á%í9🇬:🤷🔗🤦ó•😊@“”♀🤢️


In [ ]:
# import re
# def remove_urls (vTEXT):
#     vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
#     return(vTEXT)

# def remove_user(vTEXT):
#     # vTEXT = re.sub(r'(@)(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
#     vTEXT = re.sub(r'(@)(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
#     return(vTEXT)
# print( remove_user("this is a test https://sdfs.sdfsdf.com/sdfsdf/sdfsdf/sd/sdfsdfs?bob=%20tree&jef=man lets see this too https://sdfsdf.fdf.com/sdf/f end @sad213_sasd as"))

In [ ]:
from nltk import stem
import string
from gensim.parsing.preprocessing import remove_stopwords
Porter = stem.PorterStemmer()
import re
def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

def remove_user(vTEXT):
    # vTEXT = re.sub(r'(@)(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    vTEXT = re.sub(r'(@)(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

#token/stem/stopword_remove/ porcess
def data_pre(sentence_list):
  sentences = []
  # this is a-z and space
  for sentence in sentence_list:
    if sentence is np.NaN:
          sentence=''
    # create a remove table for invalid character
    remove = remove_chars
    # There are some useless char in the sentence, so here I add and delete the data
    replace = " " * len(remove)
    # make a translate table
    remove_table = str.maketrans(remove , replace)
    #remove url
    senetence_removeURL=remove_urls(sentence)
    #REMOVE USER
    senetence_removeUrlAndUser=remove_user(senetence_removeURL)
    # clean punctuation and digits
    sentence_replace = senetence_removeUrlAndUser.lower().translate(remove_table)
    #clean stopwords and tokenize
    sentence_replace = remove_stopwords(sentence_replace).split()
    #remove meaningless tokens  also remove some HTML language
    sentence_new = []
    for token in sentence_replace:
      # I think if a word are longer than 20, it have a great possiblity as a invalid word, and shorter than 2,it may be a stopword
      # so i only get the word that the length are form 2 to 20.
      if len(token) > 2 and len(token)<20:
        # lemmatizer, let the word become it's original shape, had --> have
        sentence_new.append(Porter.stem(token))
    sentences.append(sentence_new)
  return sentences


In [ ]:
# used the data_pre method that defind above
train_data = data_pre(df.text)
test_data = data_pre(df_soltion.text)

In [ ]:
print(df_soltion.text[13])
print(test_data[13])

Opinion | Pinterest takes the right step toward curbing misinformation on vaccines - The Washington Post https://t.co/8rp99QK3Jk
['opinion', 'pinterest', 'take', 'right', 'step', 'curb', 'misinform', 'vaccin', 'washington', 'post']


#  split dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data,df.tweet_label_int, test_size=0.1,
                                                    random_state=42) # so we get the same results

In [ ]:
X_train[0]

['candid',
 'senat',
 'nsw',
 'ensur',
 'jim',
 'molan',
 'vote',
 'paper',
 'go',
 'backward',
 'anti',
 'vaxxer',
 'fraser',
 'an',
 'scrape']

# Imbalence label refit

In [ ]:
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
def dummy(doc):
    return doc
make_tfidf=Pipeline([('vect', CountVectorizer(lowercase=False,tokenizer=dummy,
                    preprocessor=dummy,binary=True,ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer(use_idf=True))])

All_tfidf=make_tfidf.fit(train_data+test_data)
training_tfidf=make_tfidf.transform(X_train)
training_target=y_train


smt = SMOTE(random_state=777, k_neighbors=1)
X_SMOTE, y_SMOTE = smt.fit_sample(training_tfidf, training_target)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_SMOTE[0]

<1x86715 sparse matrix of type '<class 'numpy.float64'>'
	with 29 stored elements in Compressed Sparse Row format>

In [ ]:
val_tfidf=make_tfidf.transform(X_test)
test_tfidf=make_tfidf.transform(test_data)

In [ ]:
import collections
collections.Counter(y_SMOTE)

Counter({0: 6663, 1: 6663})

# TFIDF(SVM-NB-DT-RF-ADA-GDBT)

In [ ]:
def dummy(doc):
    return doc
svm = Pipeline([('vect', CountVectorizer(lowercase=False,tokenizer=dummy,
                    preprocessor=dummy)),
                ('tfidf', TfidfTransformer()),
                ('clf', LinearSVC(class_weight='balanced'))
               ])

nb = Pipeline([('vect', CountVectorizer(lowercase=False,tokenizer=dummy,
                    preprocessor=dummy)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())
               ])

dt = Pipeline([('vect', CountVectorizer(lowercase=False,tokenizer=dummy,
                    preprocessor=dummy)),
                ('tfidf', TfidfTransformer()),
                ('clf', DecisionTreeClassifier(class_weight='balanced'))
               ])

rf = Pipeline([('vect', CountVectorizer(lowercase=False,tokenizer=dummy,
                    preprocessor=dummy)),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier(class_weight='balanced'))
               ])
#from sklearn.ensemble import AdaBoostClassifier
adab = Pipeline([('vect', CountVectorizer(lowercase=False,tokenizer=dummy,
                    preprocessor=dummy)),
                ('tfidf', TfidfTransformer()),
                ('clf', AdaBoostClassifier())
               ])

gdbt = Pipeline([('vect', CountVectorizer(lowercase=False,tokenizer=dummy,
                    preprocessor=dummy)),
                ('tfidf', TfidfTransformer()),
                ('clf', GradientBoostingClassifier())
               ])

In [ ]:
def dummy(doc):
    return doc
vect=CountVectorizer(lowercase=False,
                    tokenizer=dummy,
                    preprocessor=dummy)
x=vect.fit_transform(train_data)
len(vect.get_feature_names())

In [ ]:

# Grid search parameters
param_grid = [{'vect__binary': [True],
               'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': [True, False],
              },
              {'vect__binary': [False],
               'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': [True, False],
              }
             ]
# Grid search parameters
param_grid_svm = [{'vect__binary': [True],
               'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': [True, False],
               'clf__C':[10,1,0.1],
               'clf__penalty':['l2']
              },
              {'vect__binary': [False],
               'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': [True, False],
               'clf__C':[10,1,0.1],
               'clf__penalty':['l2']
              }
             ]


param_grid_decision = [{'vect__binary': [True],
               'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': [True, False],
               'clf__max_depth':[2,4,8,16,32,None],
              },
              {'vect__binary': [False],
               'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': [True, False],
               'clf__max_depth':[2,4,8,16,32,None],
              }
             ]

param_grid_rf = [{'vect__binary': [True],
               'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': [True, False],
               'clf__max_depth':[2,4,8,16,32,None],
               'clf__n_estimators': [16,32,64,100],
              },
              {'vect__binary': [False],
               'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': [True, False],
               'clf__max_depth':[2,4,8,16,32,None],
               'clf__n_estimators': [16,32,64,100],
              }
             ]

param_grid_boosting = [{'vect__binary': [True],
               'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': [True, False],
               'clf__n_estimators': [64,100,150],
              },
              {'vect__binary': [False],
               'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': [True, False],
               'clf__n_estimators': [64,100,150],
              }
             ]



In [ ]:
# Find best parameters for MNB and SVM
def GridSearchCV_customed(scoreMethod=None):
    # gs_mnb_f1 = GridSearchCV(nb, param_grid,scoring=scoreMethod)
    # gs_mnb_f1.fit(X_train, y_train)
    # print('\nMNB best params:\n', gs_mnb_f1.best_params_)
    # print('\nMNB test result:\n', classification_report(y_test, gs_mnb_f1.predict(X_test)))
    # print('\nMNB train result:\n', classification_report(y_train, gs_mnb_f1.predict(X_train)))

    # gs_svm_f1 = GridSearchCV(svm, param_grid_svm,scoring=scoreMethod)
    # gs_svm_f1.fit(X_train, y_train)
    # print('\nSVM best params:\n', gs_svm_f1.best_params_)
    # print('\nSVM test result:\n', classification_report(y_test, gs_svm_f1.predict(X_test)))
    # print('\nSVM train result:\n', classification_report(y_train, gs_svm_f1.predict(X_train)))

    # dt_f1 = GridSearchCV(dt, param_grid_decision,scoring=scoreMethod)
    # dt_f1.fit(X_train, y_train)
    # print('\nDT best params:\n', dt_f1.best_params_)
    # print('\nDT test result:\n', classification_report(y_test, dt_f1.predict(X_test)))
    # print('\nDT train result:\n', classification_report(y_train, dt_f1.predict(X_train)))

    rf_f1 = GridSearchCV(rf, param_grid_rf,scoring=scoreMethod)
    rf_f1.fit(X_train, y_train)
    print('\nRF best params:\n', rf_f1.best_params_)
    print('\nRF test result:\n', classification_report(y_test, rf_f1.predict(X_test)))
    print('\nRF train result:\n', classification_report(y_train, rf_f1.predict(X_train)))


    # adab_f1 = GridSearchCV(adab, param_grid_boosting,scoring=scoreMethod)
    # adab_f1.fit(X_train, y_train)
    # print('\nAdaBoost best params:\n', adab_f1.best_params_)
    # print('\nAdaBoost test result:\n', classification_report(y_test, adab_f1.predict(X_test)))
    # print('\nAdaBoost train result:\n', classification_report(y_train, adab_f1.predict(X_train)))

    # gdbt_f1 = GridSearchCV(gdbt, param_grid_boosting,scoring=scoreMethod)
    # gdbt_f1.fit(X_train, y_train)
    # print('\nGDBoost best params:\n', gdbt_f1.best_params_)
    # print('\nGDBoost test result:\n', classification_report(y_test, gdbt_f1.predict(X_test)))
    # print('\nGDBoost train result:\n', classification_report(y_train, gdbt_f1.predict(X_train)))
    return rf_f1

## f1

In [ ]:
#F1
GridSearchCV_customed()

#SVM best params:
#  {'clf__C': 10, 'clf__penalty': 'l2', 'tfidf__use_idf': True, 'vect__binary': True, 'vect__ngram_range': (1, 2)}

# DT best params:
#  {'clf__max_depth': None, 'tfidf__use_idf': False, 'vect__binary': True, 'vect__ngram_range': (1, 1)}

# RF best params:
#  {'clf__max_depth': 32, 'clf__n_estimators': 64, 'tfidf__use_idf': True, 'vect__binary': False, 'vect__ngram_range': (1, 2)}

# AdaBoost best params:
#  {'clf__n_estimators': 64, 'tfidf__use_idf': True, 'vect__binary': False, 'vect__ngram_range': (1, 1)}

# GDBoost best params:
#  {'clf__n_estimators': 150, 'tfidf__use_idf': False, 'vect__binary': False, 'vect__ngram_range': (1, 2)}


# SVM test result:
#                precision    recall  f1-score   support

#            0       0.92      0.94      0.93       736
#            1       0.46      0.38      0.42       102

#     accuracy                           0.87       838
#    macro avg       0.69      0.66      0.67       838
# weighted avg       0.86      0.87      0.86       838

# DT test result:
#                precision    recall  f1-score   support

#            0       0.92      0.87      0.90       736
#            1       0.33      0.44      0.38       102

#     accuracy                           0.82       838
#    macro avg       0.62      0.66      0.64       838
# weighted avg       0.85      0.82      0.83       838


# RF test result:
#                precision    recall  f1-score   support

#            0       0.92      0.96      0.94       736
#            1       0.56      0.36      0.44       102

#     accuracy                           0.89       838
#    macro avg       0.74      0.66      0.69       838
# weighted avg       0.87      0.89      0.88       838



# AdaBoost test result:
#                precision    recall  f1-score   support

#            0       0.91      0.98      0.94       736
#            1       0.69      0.30      0.42       102

#     accuracy                           0.90       838
#    macro avg       0.80      0.64      0.68       838
# weighted avg       0.88      0.90      0.88       838


# GDBoost test result:
#                precision    recall  f1-score   support

#            0       0.90      0.99      0.94       736
#            1       0.75      0.21      0.32       102

#     accuracy                           0.89       838
#    macro avg       0.82      0.60      0.63       838
# weighted avg       0.88      0.89      0.87       838

## balanced_accuracy

In [ ]:

from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
# scoring_ = {'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score)}

GridSearchCV_customed(scoreMethod='balanced_accuracy')

# MNB best params:
#  {'tfidf__use_idf': True, 'vect__binary': True, 'vect__ngram_range': (1, 2)}

#  SVM best params:
#  {'clf__C': 0.1, 'clf__penalty': 'l2', 'tfidf__use_idf': False, 'vect__binary': True, 'vect__ngram_range': (1, 2)}

#  DT best params:
#  {'clf__max_depth': None, 'tfidf__use_idf': True, 'vect__binary': False, 'vect__ngram_range': (1, 1)}

#  RF best params:
#  {'clf__max_depth': 8, 'clf__n_estimators': 100, 'tfidf__use_idf': False, 'vect__binary': False, 'vect__ngram_range': (1, 1)}

# AdaBoost best params:
#  {'clf__n_estimators': 150, 'tfidf__use_idf': True, 'vect__binary': True, 'vect__ngram_range': (1, 2)}


# GDBoost best params:
#  {'clf__n_estimators': 150, 'tfidf__use_idf': True, 'vect__binary': False, 'vect__ngram_range': (1, 2)}

# SVM test result:
#                precision    recall  f1-score   support

#            0       0.95      0.85      0.90       736
#            1       0.39      0.68      0.49       102

#     accuracy                           0.83       838
#    macro avg       0.67      0.76      0.70       838
# weighted avg       0.88      0.83      0.85       838


# RF test result:
#                precision    recall  f1-score   support

#            0       0.94      0.89      0.91       736
#            1       0.42      0.59      0.49       102

#     accuracy                           0.85       838
#    macro avg       0.68      0.74      0.70       838
# weighted avg       0.88      0.85      0.86       838


# AdaBoost test result:
#                precision    recall  f1-score   support

#            0       0.91      0.95      0.93       736
#            1       0.49      0.36      0.42       102

#     accuracy                           0.88       838
#    macro avg       0.70      0.65      0.67       838
# weighted avg       0.86      0.88      0.87       838


# GDBoost test result:
#                precision    recall  f1-score   support

#            0       0.91      0.99      0.94       736
#            1       0.72      0.25      0.38       102

#     accuracy                           0.90       838
#    macro avg       0.81      0.62      0.66       838
# weighted avg       0.88      0.90      0.88       838

## jaccard

In [ ]:
rf=GridSearchCV_customed(scoreMethod='jaccard')



# MNB best params:
#  {'tfidf__use_idf': True, 'vect__binary': True, 'vect__ngram_range': (1, 1)}

#  SVM best params:
#  {'clf__C': 0.1, 'clf__penalty': 'l2', 'tfidf__use_idf': True, 'vect__binary': True, 'vect__ngram_range': (1, 2)}


# DT best params:
#  {'clf__max_depth': None, 'tfidf__use_idf': True, 'vect__binary': False, 'vect__ngram_range': (1, 1)}


# RF best params:
#  {'clf__max_depth': 16, 'clf__n_estimators': 100, 'tfidf__use_idf': True, 'vect__binary': True, 'vect__ngram_range': (1, 2)}

# AdaBoost best params:
#  {'clf__n_estimators': 150, 'tfidf__use_idf': True, 'vect__binary': True, 'vect__ngram_range': (1, 2)}

# GDBoost best params:
#  {'clf__n_estimators': 150, 'tfidf__use_idf': True, 'vect__binary': True, 'vect__ngram_range': (1, 2)}

# SVM test result:
#                precision    recall  f1-score   support

#            0       0.94      0.90      0.92       736
#            1       0.44      0.56      0.49       102

#     accuracy                           0.86       838
#    macro avg       0.69      0.73      0.70       838
# weighted avg       0.88      0.86      0.87       838


# RF test result:
#                precision    recall  f1-score   support

#            0       0.93      0.92      0.93       736
#            1       0.48      0.51      0.49       102

#     accuracy                           0.87       838
#    macro avg       0.70      0.72      0.71       838
# weighted avg       0.88      0.87      0.87       838


# AdaBoost test result:
#                precision    recall  f1-score   support

#            0       0.91      0.95      0.93       736
#            1       0.49      0.36      0.42       102

#     accuracy                           0.88       838
#    macro avg       0.70      0.65      0.67       838
# weighted avg       0.86      0.88      0.87       838


# GDBoost test result:
#                precision    recall  f1-score   support

#            0       0.90      0.98      0.94       736
#            1       0.67      0.24      0.35       102

#     accuracy                           0.89       838
#    macro avg       0.78      0.61      0.64       838
# weighted avg       0.87      0.89      0.87       838

# Stacking

In [ ]:
#  {'clf__C': 0.1, 'clf__penalty': 'l2', 'tfidf__use_idf': True, 'vect__binary': True, 'vect__ngram_range': (1, 2)}
svm = Pipeline([
                ('clf', LinearSVC(class_weight='balanced',C= 0.1, penalty= 'l2'))
               ])

#  {'clf__max_depth': 16, 'clf__n_estimators': 100, 'tfidf__use_idf': True, 'vect__binary': True, 'vect__ngram_range': (1, 2)}
rf = Pipeline([
                ('clf', RandomForestClassifier(class_weight='balanced',max_depth= 16,n_estimators=100))
               ])

#  {'clf__n_estimators': 150, 'tfidf__use_idf': True, 'vect__binary': True, 'vect__ngram_range': (1, 2)}
#from sklearn.ensemble import AdaBoostClassifier
adab = Pipeline([
                ('clf', AdaBoostClassifier(n_estimators= 150))
               ])
#  {'clf__n_estimators': 150, 'tfidf__use_idf': True, 'vect__binary': True, 'vect__ngram_range': (1, 2)}
gdbt = Pipeline([
                ('clf', GradientBoostingClassifier(n_estimators= 150))
               ])

In [ ]:
svm.fit(X_SMOTE, y_SMOTE)
svm.predict_proba(val_tfidf)

In [ ]:
X_SMOTE[0].shape

(1, 86715)

In [ ]:
from sklearn.linear_model import LogisticRegression

estimators = [svm,rf,adab,gdbt]
stack_model = LogisticRegression()
stk = Stacking(estimators,stack_model,use_prob=False,n_splits=5,verbose=1)
stk.fit(X_SMOTE, y_SMOTE)
y_pred = stk.predict(val_tfidf)

print('\nstacking test result:\n', classification_report(y_test, stk.predict(val_tfidf)))

Base Estimators are  Pipeline(memory=None,
         steps=[('clf',
                 LinearSVC(C=0.1, class_weight='balanced', dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='squared_hinge', max_iter=1000,
                           multi_class='ovr', penalty='l2', random_state=None,
                           tol=0.0001, verbose=0))],
         verbose=False) Pipeline(memory=None,
         steps=[('clf',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight='balanced',
                                        criterion='gini', max_depth=16,
                                        max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_sampl

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

'''
Stacking classifier
Created on 3/4/2107
An ensemble-learning meta-classifier for stacking
Author: Kaushal Shetty
'''

import numpy as np
from copy import copy
from sklearn.model_selection import KFold


class Stacking(object):

    def __init__(
        self,
        estimators,
        stacked_model,
        use_prob=False,
        n_splits=5,
        verbose=0,
        ):
        """
............Stacked Generalizer Classifier
............Trains a series of base models using K-fold cross-validation, then combines
............the predictions of each model into a set of features that are used to train
............a high-level classifier model. 
............Parameters
............-----------
............estmitators: list of classifier models
................Each model must have a .fit and .predict_proba/.predict method a'la
................sklearn
............stacked_model: object
................A classifier model used to aggregate the outputs of the trained base
................models. Must have a .fit and .predict_proba/.predict method
............use_prob: boolean 
................If True takes predict_proba as features else takes predict as features.
............n_splis: int
................The number of K-folds to use in =cross-validated model training
............verbose: 0 or 1
............
........"""

        super(Stacking, self).__init__()
        if len(estimators) == 0 or stacked_model is None:
            raise ValueError('Length of estimators must be greater than 0  and stack model connot be None type'
                             )

        if use_prob:
            for i in range(len(estimators)):
                if not hasattr(estimators[i], 'predict_proba'):
                    raise AttributeError(estimators[i],
                            ' has no attribute predict_proba. Either set use_prob to False or use a classifier that supports predict_proba'
                            )
        self.k = n_splits
        self.estimators = estimators
        self.golbal_x = None
        self.global_y = None
        self.stackModel = stacked_model
        self.global_stack = None
        self.verbose = verbose
        self.use_prob = use_prob

    def fit(self, x, y):
        """
    ........
    ........----------
    ........estimators : list of base models, shape = len(estimators)
    ........stacked_model : meta ensembling model (level 2 classifier)
    ........x :  {array-like}, shape = [n_samples, n_features selected]
................ Training vectors, where n_samples is the number of samples and n_features is the number of features.
 
    ........y  : array-like, shape = [n_samples]
    ............ Target Values
    ........Returns
    ........-------
    ........self : object 
    ...."""

        if self.verbose > 0:
            print ('Base Estimators are ', self.estimators[0],
                   self.estimators[1])
            print ('Stacked model is', self.stackModel)

        self.global_x = x
        self.global_y = y
        kf = KFold(n_splits=self.k)
        X_stack = None
        cnt = 0
        print( self.use_prob)
        for (train_index, test_index) in kf.split(x, y):
            cnt = cnt + 1

            if self.verbose > 0:
                print (cnt, ' iteration')
                print ('Fitting...')

            (X_train, X_test) = (x[train_index], x[test_index])
            (y_train, y_test) = (y[train_index], y[test_index])
            X_test_new_feats = None

            for (i, estimator) in enumerate(self.estimators):
                estimator.fit(X_train, y_train)

                if self.use_prob:
                    if i == 0:
                        X_test_new_feats = np.column_stack((X_test,
                                np.asarray(estimator.predict_proba(X_test),
                                dtype=float)))
                    else:
                        X_test_new_feats = \
                            np.column_stack((X_test_new_feats,
                                np.asarray(estimator.predict_proba(X_test),
                                dtype=float)))
                else:

                    if i == 0:
                        # print(X_test.shape)
                        # print(np.asarray(estimator.predict(X_test)).shape)
                        # X_test_new_feats = np.column_stack((X_test,
                        #         np.asarray(estimator.predict(X_test),
                        #         dtype=float)))
                        X_test_new_feats = np.asarray(estimator.predict(X_test))
                    else:
                        
                        X_test_new_feats = \
                            np.column_stack((X_test_new_feats,
                                np.asarray(estimator.predict(X_test),
                                dtype=float)))

            if X_stack is None:

                X_stack = X_test_new_feats
            else:
                X_stack = np.vstack((X_stack, X_test_new_feats))

        if self.verbose > 0:
            print ('Model Fitted.')

        model = copy(self.stackModel)
        print (X_stack.shape)
        self.global_stack = X_stack

        return self

    def predict(self, x_test):
        """ 
............Predict target values for X.
............Parameters
................x_test : {array-like}, shape = [n_samples, n_features]
................ vectors, where n_samples is the number of samples and n_features is the number of features.
............Returns
................labels : array-like, shape = [n_samples]
................Predicted class labels.

........"""

        if self.verbose > 0:
            print ('Predicting....')

        for (i, estimator) in enumerate(self.estimators):
            estimator.fit(self.global_x, self.global_y)
            if self.use_prob:
                if i == 0:
                    x_test_meta = np.column_stack((x_test,
                            np.asarray(estimator.predict_proba(x_test),
                            dtype=float)))
                else:
                    x_test_meta = np.column_stack((x_test_meta,
                            np.asarray(estimator.predict_proba(x_test),
                            dtype=float)))
            else:

                if i == 0:
                    # x_test_meta = np.column_stack((x_test,
                    #         np.asarray(estimator.predict(x_test),
                    #         dtype=float)))
                    x_test_meta = np.asarray(estimator.predict(x_test))
                else:
                    x_test_meta = np.column_stack((x_test_meta,
                            np.asarray(estimator.predict(x_test),
                            dtype=float)))

        model = copy(self.stackModel)
        model.fit(self.global_stack, self.global_y)

        if self.verbose > 0:
            print( 'Done.')

        return np.asarray(model.predict(x_test_meta))


# Output

In [ ]:


rf_out = Pipeline([
                ('clf', RandomForestClassifier())
               ])


from sklearn.model_selection import StratifiedKFold
# RF best params:
#  {'clf__max_depth': 16, 'clf__n_estimators': 100, 'tfidf__use_idf': True, 'vect__binary': True, 'vect__ngram_range': (1, 2)}
param_grid_rf = [{
               'clf__max_depth':[16],
               'clf__n_estimators': [100,150,200],
              },
            
             ]

# cvKFold=StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
def goTraining(model,para,scoring=None):
    rf_f1 = GridSearchCV(model, para,scoring=scoring,cv=10)
    rf_f1.fit(X_SMOTE, y_SMOTE)
    print('\n best params:\n', rf_f1.best_params_)
    print('\n test result:\n', classification_report(y_test, rf_f1.predict(val_tfidf)))
    print('\n train result:\n', classification_report(y_SMOTE, rf_f1.predict(X_SMOTE)))
    return rf_f1

rf_f1=goTraining(rf_out,param_grid_rf)

In [ ]:
svm_out = Pipeline([
               ('clf', LinearSVC(class_weight='balanced'))
               ])
param_grid_svm = [{
               'clf__C':[10,1,0.1],
               'clf__penalty':['l2']
              }
            ]
svm_f1=goTraining(svm_out,param_grid_svm,'balanced_accuracy')

In [ ]:
#Nevertheless, lets create the final output data to upload up to kaggle for submission
#Test our classifier on the hydrated target tweets
def outputFile(gs_svm,df_soltion):
    # df_soltion = df_soltion.replace(np.nan, '', regex=True)
    predicted_targets = gs_svm.predict(test_tfidf)
    #Set the solution to our predicted targets
    df_soltion.tweet_label_int=predicted_targets
    df_soltion_out=df_soltion[['tweetid','tweet_label_int']]
    #And save it out to upload to competition!
    df_soltion_out.to_csv("my_kaggle_solution.csv",index=False)

In [ ]:
outputFile(stk,df_soltion)

Predicting....
Done.


# Embeding

In [ ]:
# text_train_ns
# Data after word segmentation has been processed
len_list = [len(s) for s in train_data]
#seq_length，Is the length of the sublist with the most words.
seq_length = max(len_list)

# add_padding
# The function of the function is to fill in the blanks, 
# the purpose is to fill the length of all the sublists to a consistent length, 
# which uses <PAD> as the filling content.
def add_padding(corpus, seq_length):
    output = []
    for sentence in corpus:
        if len(sentence)>seq_length:
            output.append(sentence[:seq_length])
        else:
            for j in range(seq_length-len(sentence)):
                sentence.append("<PAD>")
            output.append(sentence)
    return output

text_train_pad = add_padding(train_data,seq_length )
text_test_pad = add_padding(test_data,seq_length )
print(len(text_test_pad))

906


In [ ]:
text_test_pad[0]

['true',
 'day',
 'thing',
 'stand',
 'larg',
 'illeg',
 'come',
 'countri',
 'vaccin',
 'kid',
 'diseas',
 'mutat',
 'ignor',
 'folk',
 'think',
 'vaccin',
 'caus',
 'autism',
 'shouldn',
 'forc',
 'requir',
 'school',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>']

In [ ]:
seq_length

40

In [ ]:
import gensim.downloader as api
word_emb_model = api.load("glove-twitter-100")

[================================================--] 97.3% 376.6/387.1MB downloaded

In [ ]:
def get_embeddings(corpus,word_emb_model):
    emb_dim = word_emb_model.vector_size
    out = []
    for sentence in corpus:
        out_temp = []
        for word in sentence:
            try:
                out_temp.append(word_emb_model.wv[word])
            except:
                out_temp.append([0]*emb_dim)
    
        out.append(out_temp)
    return np.array(out)


#embeddings
train_emb = get_embeddings(text_train_pad,word_emb_model)
test_emb = get_embeddings(text_test_pad,word_emb_model)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [ ]:
train_emb.shape

(8372, 40, 100)

# Embeding + LSTM/GRU

In [ ]:
def labelToneHot(item):
    label=item
    temp=np.zeros(2)
    temp[label]=1
    return temp[:]
Target=[]
for item in df.tweet_label_int:
    Target.append(labelToneHot(item))


In [ ]:
Target[:10]

[array([1., 0.]),
 array([1., 0.]),
 array([1., 0.]),
 array([0., 1.]),
 array([1., 0.]),
 array([1., 0.]),
 array([1., 0.]),
 array([1., 0.]),
 array([1., 0.]),
 array([1., 0.])]

In [ ]:
sent_train, sent_val, label_train, label_val = train_test_split(train_emb, np.array(Target), test_size=0.2, random_state=42)

In [ ]:
sent_train.shape


(6697, 40, 100)

In [ ]:
sent_train[0]

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.093424  , -0.44723001, -0.16154   , ...,  0.2397    ,
         0.32007   ,  0.23650999],
       [ 0.091552  ,  0.093336  , -0.028113  , ...,  0.33173001,
        -0.087436  , -0.55506003],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

##Imbalance issue refit

In [ ]:
smt = SMOTE(random_state=777, k_neighbors=1)
X_SMOTE, y_SMOTE = smt.fit_sample(sent_train.reshape(-1,4000), label_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
y_SMOTE_binary=[]
for item in y_SMOTE:
    y_SMOTE_binary.append(labelToneHot(item))

In [ ]:
y_SMOTE_binary[0:10]

[array([1., 0.]),
 array([0., 1.]),
 array([1., 0.]),
 array([1., 0.]),
 array([0., 1.]),
 array([0., 1.]),
 array([1., 0.]),
 array([1., 0.]),
 array([0., 1.]),
 array([1., 0.])]

In [ ]:
X_SMOTE=X_SMOTE.reshape(-1,40,100)

In [ ]:
X_SMOTE[0]

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.093424  , -0.44723001, -0.16154   , ...,  0.2397    ,
         0.32007   ,  0.23650999],
       [ 0.091552  ,  0.093336  , -0.028113  , ...,  0.33173001,
        -0.087436  , -0.55506003],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

## LSTM

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
class trainset(Dataset):
    def __init__(self,senetance,target):
        
        self.senetance = senetance
        self.target = target

    def __getitem__(self, index):
        
        senetance = self.senetance[index]
        target = self.target[index]
        return senetance,target

    def __len__(self):
        return len(self.senetance)

#create dataloader
train_data = trainset(senetance=X_SMOTE,target=y_SMOTE_binary)
trainloader = DataLoader(train_data, batch_size =2, shuffle = True, num_workers=6,pin_memory=True,drop_last=True)
validate_data= trainset(senetance=sent_val,target=label_val)
validateloader = DataLoader(validate_data, batch_size =2, shuffle = True, num_workers=6,pin_memory=True,drop_last=True)

In [ ]:
#calculate the accuracy and recall
def calculate_acuracy_mode_one(model_pred, labels):
    accuracy_th = 0.5
    pred_result = model_pred > accuracy_th
    pred_result = pred_result.float()
    pred_one_num = torch.sum(pred_result)
    if pred_one_num == 0:
        return 0.0001, 0.0001
    target_one_num = torch.sum(labels)
    true_predict_num = torch.sum(pred_result * labels)
    precision = true_predict_num / pred_one_num
    recall = true_predict_num / target_one_num
    return precision.item(), recall.item()

In [ ]:
#define validation function
def evaluation_lstm(model): 
        if torch.cuda.is_available():
            model = model.to('cuda')
            print('using GPU')
        else:
            print('using CPU')

        ## set model to evaluation mode
        model.eval()
        ## evaluation
        ep_loss = 0.0
        ep_acc = 0.0
        ep_recall=0.0
        criterion=nn.BCEWithLogitsLoss()
        with torch.no_grad():
            for step,(x,y) in enumerate(validateloader):
                x=x.float()
                y=y.float()
                if torch.cuda.is_available():
                    x = x.to('cuda')
                    y = y.to('cuda')
                # calculate output
                p = model(x)
                # calculate metrics
                loss = criterion(p, y)

                pred = F.sigmoid(p)
                target = (y)
                acc ,recall= calculate_acuracy_mode_one(pred,target)
                # acc = accuracy(p, y)
                ep_loss += loss
                ep_acc += acc
                ep_recall+=recall
                if step % 5 == 0:
                    print('\rtest step: %d loss: %.2f acc: %.4f recall: %.4f F1: %.4f'
                            % (step, ep_loss/(step+1), ep_acc/(step+1),ep_recall/(step+1), 
                             2*ep_acc/(step+1)*ep_recall/(step+1)/(ep_acc/(step+1)+ep_recall/(step+1))
                          ),end='')
        print('\ntest loss: %.2f acc: %.4f recall: %.4f F1: %.4f'
                % (ep_loss/(step+1), ep_acc/(step+1),ep_recall/(step+1),2*ep_acc/(step+1)*ep_recall/(step+1)/(ep_acc/(step+1)+ep_recall/(step+1))))
        

#define validation function
def evaluation_gru(model): 
        if torch.cuda.is_available():
            model = model.to('cuda')
            print('using GPU')
        else:
            print('using CPU')

        ## set model to evaluation mode
        model.eval()
        ## evaluation
        ep_loss = 0.0
        ep_acc = 0.0
        ep_recall=0.0
        criterion=nn.BCEWithLogitsLoss()
        with torch.no_grad():
            for step,(x,y) in enumerate(validateloader):
                x=x.float()
                y=y.float()
                if torch.cuda.is_available():
                    x = x.to('cuda')
                    y = y.to('cuda')
                h = model.init_hidden(16)
                # calculate output
                p,h = model(x,h)
                # calculate metrics
                loss = criterion(p, y)

                pred = F.sigmoid(p)
                target = (y)
                acc ,recall= calculate_acuracy_mode_one(pred,target)
                # acc = accuracy(p, y)
                ep_loss += loss
                ep_acc += acc
                ep_recall+=recall
                if step % 5 == 0:
                    print('\rtest step: %d loss: %.2f acc: %.4f recall: %.4f F1: %.4f'
                            % (step, ep_loss/(step+1), ep_acc/(step+1),ep_recall/(step+1), 
                             2*ep_acc/(step+1)*ep_recall/(step+1)/(ep_acc/(step+1)+ep_recall/(step+1))
                          ),end='')
        print('\ntest loss: %.2f acc: %.4f recall: %.4f F1: %.4f'
                % (ep_loss/(step+1), ep_acc/(step+1),ep_recall/(step+1),2*ep_acc/(step+1)*ep_recall/(step+1)/(ep_acc/(step+1)+ep_recall/(step+1))))

In [ ]:
from torch import nn
n_input = train_emb.shape[2]
n_hidden = 100
n_class = 2
num_layers= 2
total_epoch = 20
learning_rate = 0.01
class Lstm_Net_2(nn.Module):
    def __init__(self):
        super(Lstm_Net_2, self).__init__()
        self.lstm = nn.LSTM(n_input, n_hidden, num_layers,batch_first =True,bidirectional=True, dropout=0.3)
        self.linear = nn.Linear(n_hidden*2,n_class)

    def forward(self, sentence):
        #h_n of shape (num_layers * num_directions, batch, hidden_size): tensor containing the hidden state for t = seq_len.
        lstm_out, (h_n,c_n) = self.lstm(sentence)
        #concat the last hidden state from two direction
        hidden_out =torch.cat((h_n[0,:,:],h_n[1,:,:]),1)#concatnate as colunm(left-right)
        z = self.linear(hidden_out)

        return z

In [ ]:
class GRUNet(nn.Module):
    def __init__(self, input_dim=train_emb.shape[2], hidden_dim=100, output_dim=2, n_layers=2, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

class LSTMNet(nn.Module):
    def __init__(self, input_dim=train_emb.shape[2], hidden_dim=100, output_dim=2, n_layers=2, drop_prob=0.2):
        super(LSTMNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.lstm(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [ ]:

#define training function
def train_lstm(blstm):
    epoch = 2
    learning_rate = 0.001
    device = torch.device("cuda" if  torch.cuda.is_available() else "cpu")
    # print(device)

    if torch.cuda.is_available():
        blstm = blstm.to('cuda')
        print('using GPU')
    else:
        print('using CPU')

    criterion = nn.BCEWithLogitsLoss()

    optimizer = torch.optim.Adam(blstm.parameters(), lr=learning_rate)
    # optimizer = torch.optim.SGD(blstm.parameters(), learning_rate, momentum=0.9,weight_decay =1e-10)
    # optimizer = torch.optim.Adam(model.parameters(), learning_rate)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    batch_num = 0
    blstm.train()
    for ep in range(epoch):
        ep_loss = 0.0
        ep_acc = 0.0
        ep_recall=0.0
        for batch,(x,y) in enumerate(trainloader):
            x=x.float()
            y=y.float()
            if torch.cuda.is_available():
                x = x.to('cuda')
                y = y.to('cuda')
            
            optimizer.zero_grad()
            outputs = blstm(x)
            # print(type(outputs[0]))
            # print(type(y[0]))
            # 1/0
            loss = criterion(outputs,y)
            # print(outputs)
            # print(y)
            a=outputs
            b=y
            loss.backward()
            optimizer.step()
            #batch_num += 1

            pred = F.sigmoid(outputs)
            target = (y)
            # print('pred:',pred)
            # print('target:',target)
            acc,recall = calculate_acuracy_mode_one(pred,y)
            # acc = accuracy(p, y)
            # print(calculate_acuracy_mode_one(outputs,y))
            # loss=(loss*weight).mean()
            ep_loss += loss
            ep_acc += acc
            ep_recall+=recall
            if batch % 100 == 0:
                # print((ep_acc/(batch+1)+ep_recall/(batch+1)))
                print('\rtrain step: %d loss: %.2f acc: %.4f recall: %.4f F1: %.4f' 
                            % (batch, ep_loss/(batch+1), ep_acc/(batch+1),ep_recall/(batch+1),
                               2*ep_acc/(batch+1)*ep_recall/(batch+1)/(ep_acc/(batch+1)+ep_recall/(batch+1))
                               ), end='')
        exp_lr_scheduler.step()
        print('\nepoch: %d loss: %.2f acc: %.4f recall: %.4f F1: %.4f'
            % (ep, ep_loss/(batch+1), ep_acc/(batch+1),ep_recall/(batch+1),
               2*ep_acc/(batch+1)*ep_recall/(batch+1)/(ep_acc/(batch+1)+ep_recall/(batch+1))
               ))
        print('Validation:')
        evaluation_lstm(blstm)
        print('\n')
        blstm.train()
    return blstm
        

In [ ]:

#define training function
def train_gru(gru):
    epoch = 5
    learning_rate = 0.001
    batch_size=2
    device = torch.device("cuda" if  torch.cuda.is_available() else "cpu")
    # print(device)

    if torch.cuda.is_available():
        gru = gru.to('cuda')
        print('using GPU')
    else:
        print('using CPU')

    criterion = nn.BCEWithLogitsLoss()

    optimizer = torch.optim.Adam(gru.parameters(), lr=learning_rate)
    # optimizer = torch.optim.SGD(gru.parameters(), learning_rate, momentum=0.9,weight_decay =1e-10)
    # optimizer = torch.optim.Adam(model.parameters(), learning_rate)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    batch_num = 0
    gru.train()
    for ep in range(epoch):
        h = gru.init_hidden(batch_size)
        ep_loss = 0.0
        ep_acc = 0.0
        ep_recall=0.0
        for batch,(x,y) in enumerate(trainloader):
            x=x.float()
            y=y.float()
            if torch.cuda.is_available():
                x = x.to('cuda')
                y = y.to('cuda')
            h = h.data
            # h = tuple([e.data for e in h])
            optimizer.zero_grad()
            outputs,h = gru(x,h)
            # print(type(outputs[0]))
            # print(type(y[0]))
            # 1/0
            loss = criterion(outputs,y)
            # print(outputs)
            # print(y)
            a=outputs
            b=y
            loss.backward()
            optimizer.step()
            #batch_num += 1

            # pred = F.sigmoid(outputs)
            target = (y)
            # print('pred:',pred)
            # print('target:',target)
            acc,recall = calculate_acuracy_mode_one(outputs,y)
            # acc = accuracy(p, y)
            # print(calculate_acuracy_mode_one(outputs,y))
            # loss=(loss*weight).mean()
            ep_loss += loss
            ep_acc += acc
            ep_recall+=recall
            if batch % 100 == 0:
                # print((ep_acc/(batch+1)+ep_recall/(batch+1)))
                print('\rtrain step: %d loss: %.2f acc: %.4f recall: %.4f F1: %.4f' 
                            % (batch, ep_loss/(batch+1), ep_acc/(batch+1),ep_recall/(batch+1),
                               2*ep_acc/(batch+1)*ep_recall/(batch+1)/(ep_acc/(batch+1)+ep_recall/(batch+1))
                               ), end='')
        exp_lr_scheduler.step()
        print('\nepoch: %d loss: %.2f acc: %.4f recall: %.4f F1: %.4f'
            % (ep, ep_loss/(batch+1), ep_acc/(batch+1),ep_recall/(batch+1),
               2*ep_acc/(batch+1)*ep_recall/(batch+1)/(ep_acc/(batch+1)+ep_recall/(batch+1))
               ))
        print('Validation:')
        evaluation_gru(gru)
        print('\n')
        gru.train()
    return gru
        

In [ ]:
#training model
# gru
from torch.optim import lr_scheduler
import torch
device = torch.device("cuda" if  torch.cuda.is_available() else "cpu")
gru_2 = GRUNet().to(device)
gru_2 = train_gru(gru_2)

In [ ]:
# lstm
lstm_2 = Lstm_Net_2().to(device)
lstm_2 = train_lstm(lstm_2)

using GPU


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


train step: 2900 loss: 0.44 acc: 0.7975 recall: 0.7952 F1: 0.7963
epoch: 0 loss: 0.44 acc: 0.7997 recall: 0.7974 F1: 0.7985
Validation:
using GPU
test step: 415 loss: 0.38 acc: 0.8417 recall: 0.8413 F1: 0.8415
test loss: 0.38 acc: 0.8425 recall: 0.8421 F1: 0.8423


train step: 2900 loss: 0.24 acc: 0.9030 recall: 0.9025 F1: 0.9027
epoch: 1 loss: 0.24 acc: 0.9035 recall: 0.9031 F1: 0.9033
Validation:
using GPU
test step: 415 loss: 0.39 acc: 0.8495 recall: 0.8510 F1: 0.8502
test loss: 0.39 acc: 0.8496 recall: 0.8511 F1: 0.8504




In [ ]:
torch.save(gru_2.state_dict(), 'gru')

In [ ]:
gru_2 = GRUNet().to(device)
gru_2.load_state_dict(torch.load('gru'))
gru_2.eval()

GRUNet(
  (gru): GRU(100, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=2, bias=True)
  (relu): ReLU()
)

In [ ]:
def make_prediction_val(blstm):
    blstm.eval()
    with torch.no_grad():
        test_data = torch.from_numpy(sent_val).to('cuda').float()
        output_predicted = blstm(test_data)
        output_predicted = F.sigmoid(output_predicted)
        print(output_predicted)
        accuracy_rat = 0.5
        pre_result = output_predicted  >= accuracy_rat
        pre_result = pre_result.float()
    return pre_result
val_data=make_prediction_val(lstm_2)
pred_val=vector_to_label_lstm(val_data)
label_val_int=vector_to_label_lstm(label_val)
print('\nLSTM val result:\n', classification_report(label_val_int, pred_val))

# LSTM val result:
#                precision    recall  f1-score   support

#            0       0.91      0.93      0.92      1458
#            1       0.42      0.36      0.39       217

#     accuracy                           0.85      1675
#    macro avg       0.66      0.64      0.65      1675
# weighted avg       0.84      0.85      0.85      1675


def make_prediction_val_gru(blstm):
    blstm.eval()
    with torch.no_grad():
        test_data = torch.from_numpy(sent_val).to('cuda').float()
        h = blstm.init_hidden(test_data.shape[0])
        output_predicted,h = blstm(test_data,h)
        output_predicted = F.sigmoid(output_predicted)
        accuracy_rat = 0.5
        pre_result = output_predicted  >= accuracy_rat
        pre_result = pre_result.float()
    return pre_result

tensor([[0.9736, 0.0278],
        [0.8249, 0.1887],
        [0.8874, 0.1227],
        ...,
        [0.6691, 0.3230],
        [0.9351, 0.0627],
        [0.9681, 0.0307]], device='cuda:0')

LSTM val result:
               precision    recall  f1-score   support

           0       0.90      0.93      0.91      1458
           1       0.40      0.31      0.35       217

    accuracy                           0.85      1675
   macro avg       0.65      0.62      0.63      1675
weighted avg       0.84      0.85      0.84      1675



/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
test_data = torch.from_numpy(sent_val).to('cuda').float()
output_predicted = lstm_2(test_data)

In [ ]:
print(output_predicted)
accuracy_rat = 0.5
pre_result = output_predicted  >= accuracy_rat
pre_result = pre_result.float()
print(pre_result)

tensor([[ 3.3396, -3.3170],
        [-0.8554,  0.8166],
        [ 3.0410, -3.1327],
        ...,
        [ 2.3221, -2.2448],
        [ 3.1789, -3.1770],
        [ 2.8899, -2.8717]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[1., 0.],
        [0., 1.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]], device='cuda:0')


In [ ]:
#gru
val_data=make_prediction_val_gru(gru_2)
pred_val=vector_to_label_lstm(val_data)
label_val_int=vector_to_label_lstm(label_val)
print('\nLSTM val result:\n', classification_report(label_val_int, pred_val))


LSTM val result:
               precision    recall  f1-score   support

           0       0.89      0.95      0.92      1458
           1       0.40      0.24      0.30       217

    accuracy                           0.86      1675
   macro avg       0.65      0.59      0.61      1675
weighted avg       0.83      0.86      0.84      1675



## output

In [ ]:
# predict the test data
def make_prediction(blstm):
    blstm.eval()
    with torch.no_grad():
        test_data = torch.from_numpy(test_emb).to('cuda').float()
        output_predicted = blstm(test_data)
        output_predicted = F.sigmoid(output_predicted)
        accuracy_rat = 0.5
        pre_result = output_predicted  >= accuracy_rat
        pre_result = pre_result.float()
    return pre_result
# transfer vector to label
def vector_to_label_lstm(vec):
    output=[]
    
    for label in vec:
        flag=0
        if label[0]==1:
            flag=0
        else:
            flag=1
        output.append(flag)
    return output
#make kaggle file
def outputFile(blstm):
    pre_result = make_prediction(blstm)    
    pre_lsit=vector_to_label_lstm(pre_result)
    # create test file for kaggle
    df_soltion.tweet_label_int=pre_lsit
    df_soltion_out=df_soltion[['tweetid','tweet_label_int']]
    #And save it out to upload to competition!
    df_soltion_out.to_csv("my_kaggle_solution_lstm.csv",index=False)

In [ ]:
pre_result = make_prediction(blstm_2)    
pre_lsit=vector_to_label_lstm(pre_result)

In [ ]:
outputFile(blstm_2)

In [ ]:
pre_result[:10]

tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.]], device='cuda:0')

# Attention

In [ ]:
# _*_ coding: utf-8 _*_

import torch
from torch.optim import lr_scheduler
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F

class SelfAttention(nn.Module):
	def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_length, weights):
		super(SelfAttention, self).__init__()

		"""
		Arguments
		---------
		batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
		output_size : 2 = (pos, neg)
		hidden_sie : Size of the hidden_state of the LSTM
		vocab_size : Size of the vocabulary containing unique words
		embedding_length : Embeddding dimension of GloVe word embeddings
		weights : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table 
		
		--------
		
		"""

		self.batch_size = batch_size
		self.output_size = output_size
		self.hidden_size = hidden_size
		self.vocab_size = vocab_size
		self.embedding_length = embedding_length
		self.weights = weights

		self.word_embeddings = nn.Embedding(vocab_size, embedding_length)
		self.word_embeddings.weights = nn.Parameter(weights, requires_grad=False)
		self.dropout = 0.8
		self.bilstm = nn.LSTM(embedding_length, hidden_size, dropout=self.dropout, bidirectional=True)
		# We will use da = 350, r = 30 & penalization_coeff = 1 as per given in the self-attention original ICLR paper
		self.W_s1 = nn.Linear(2*hidden_size, 350)
		self.W_s2 = nn.Linear(350, 30)
		self.fc_layer = nn.Linear(30*2*hidden_size, 2000)
		self.label = nn.Linear(2000, output_size)

	def attention_net(self, lstm_output):

		"""
		Now we will use self attention mechanism to produce a matrix embedding of the input sentence in which every row represents an
		encoding of the inout sentence but giving an attention to a specific part of the sentence. We will use 30 such embedding of 
		the input sentence and then finally we will concatenate all the 30 sentence embedding vectors and connect it to a fully 
		connected layer of size 2000 which will be connected to the output layer of size 2 returning logits for our two classes i.e., 
		pos & neg.
		Arguments
		---------
		lstm_output = A tensor containing hidden states corresponding to each time step of the LSTM network.
		---------
		Returns : Final Attention weight matrix for all the 30 different sentence embedding in which each of 30 embeddings give
				  attention to different parts of the input sentence.
		Tensor size : lstm_output.size() = (batch_size, num_seq, 2*hidden_size)
					  attn_weight_matrix.size() = (batch_size, 30, num_seq)
		"""
		attn_weight_matrix = self.W_s2(F.tanh(self.W_s1(lstm_output)))
		attn_weight_matrix = attn_weight_matrix.permute(0, 2, 1)
		attn_weight_matrix = F.softmax(attn_weight_matrix, dim=2)

		return attn_weight_matrix

	def forward(self, input_sentences, batch_size=None):

		""" 
		Parameters
		----------
		input_sentence: input_sentence of shape = (batch_size, num_sequences)
		batch_size : default = None. Used only for prediction on a single sentence after training (batch_size = 1)
		
		Returns
		-------
		Output of the linear layer containing logits for pos & neg class.
		
		"""

		# input = self.word_embeddings(input_sentences)
		input = input_sentences.permute(1, 0, 2)
		if batch_size is None:
			h_0 = Variable(torch.zeros(2, self.batch_size, self.hidden_size).cuda())
			c_0 = Variable(torch.zeros(2, self.batch_size, self.hidden_size).cuda())
		else:
			h_0 = Variable(torch.zeros(2, batch_size, self.hidden_size).cuda())
			c_0 = Variable(torch.zeros(2, batch_size, self.hidden_size).cuda())

		output, (h_n, c_n) = self.bilstm(input, (h_0, c_0))
		output = output.permute(1, 0, 2)
		# output.size() = (batch_size, num_seq, 2*hidden_size)
		# h_n.size() = (1, batch_size, hidden_size)
		# c_n.size() = (1, batch_size, hidden_size)
		attn_weight_matrix = self.attention_net(output)
		# attn_weight_matrix.size() = (batch_size, r, num_seq)
		# output.size() = (batch_size, num_seq, 2*hidden_size)
		hidden_matrix = torch.bmm(attn_weight_matrix, output)
		# hidden_matrix.size() = (batch_size, r, 2*hidden_size)
		# Let's now concatenate the hidden_matrix and connect it to the fully connected layer.
		fc_out = self.fc_layer(hidden_matrix.view(-1, hidden_matrix.size()[1]*hidden_matrix.size()[2]))
		logits = self.label(fc_out)
		# logits.size() = (batch_size, output_size)

		return logits

In [ ]:
def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)
def train_attn(attn):
    epoch = 1
    learning_rate = 0.001
    device = torch.device("cuda" if  torch.cuda.is_available() else "cpu")
    # print(device)

    if torch.cuda.is_available():
        attn = attn.to('cuda')
        print('using GPU')
    else:
        print('using CPU')

    criterion = nn.BCEWithLogitsLoss()

    optimizer = torch.optim.Adam(attn.parameters(), lr=learning_rate)
    # optimizer = torch.optim.SGD(attn.parameters(), learning_rate, momentum=0.9,weight_decay =1e-10)
    # optimizer = torch.optim.Adam(model.parameters(), learning_rate)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    batch_num = 0
    attn.train()
    for ep in range(epoch):
        ep_loss = 0.0
        ep_acc = 0.0
        ep_recall=0.0
        for batch,(x,y) in enumerate(trainloader):
            x=x.float()
            y=y.float()
            if torch.cuda.is_available():
                x = x.to('cuda')
                y = y.to('cuda')
            
            optimizer.zero_grad()
            outputs = attn(x)
            # print(type(outputs[0]))
            # print(type(y[0]))
            # 1/0
            loss = criterion(outputs,y)
            # print(outputs)
            # print(y)
            a=outputs
            b=y
            loss.backward()

            clip_gradient(attn, 1e-1)

            optimizer.step()
            #batch_num += 1

            pred = F.sigmoid(outputs)
            target = (y)
            # print('pred:',pred)
            # print('target:',target)
            acc,recall = calculate_acuracy_mode_one(pred,target)
            # acc = accuracy(p, y)
            # print(calculate_acuracy_mode_one(outputs,y))
            # loss=(loss*weight).mean()
            ep_loss += loss
            ep_acc += acc
            ep_recall+=recall
            if batch % 100 == 0:
                # print((ep_acc/(batch+1)+ep_recall/(batch+1)))
                print('\rtrain step: %d loss: %.2f acc: %.4f recall: %.4f F1: %.4f' 
                            % (batch, ep_loss/(batch+1), ep_acc/(batch+1),ep_recall/(batch+1),
                               2*ep_acc/(batch+1)*ep_recall/(batch+1)/(ep_acc/(batch+1)+ep_recall/(batch+1))
                               ), end='')
        exp_lr_scheduler.step()
        print('\nepoch: %d loss: %.2f acc: %.4f recall: %.4f F1: %.4f'
            % (ep, ep_loss/(batch+1), ep_acc/(batch+1),ep_recall/(batch+1),
               2*ep_acc/(batch+1)*ep_recall/(batch+1)/(ep_acc/(batch+1)+ep_recall/(batch+1))
               ))
        print('Validation:')
        evaluation_attn(attn)
        print('\n')
        attn.train()
    return attn

def evaluation_attn(model): 
        if torch.cuda.is_available():
            model = model.to('cuda')
            print('using GPU')
        else:
            print('using CPU')

        ## set model to evaluation mode
        model.eval()
        ## evaluation
        ep_loss = 0.0
        ep_acc = 0.0
        ep_recall=0.0
        criterion=nn.BCEWithLogitsLoss()
        with torch.no_grad():
            for step,(x,y) in enumerate(validateloader):
                x=x.float()
                y=y.float()
                if torch.cuda.is_available():
                    x = x.to('cuda')
                    y = y.to('cuda')
                # calculate output
                p = model(x)
                # calculate metrics
                loss = criterion(p, y)

                pred = F.sigmoid(p)
                target = (y)
                acc ,recall= calculate_acuracy_mode_one(pred,target)
                # acc = accuracy(p, y)
                ep_loss += loss
                ep_acc += acc
                ep_recall+=recall
                if step % 5 == 0:
                    print('\rtest step: %d loss: %.2f acc: %.4f recall: %.4f F1: %.4f'
                            % (step, ep_loss/(step+1), ep_acc/(step+1),ep_recall/(step+1), 
                             2*ep_acc/(step+1)*ep_recall/(step+1)/(ep_acc/(step+1)+ep_recall/(step+1))
                          ),end='')
        print('\ntest loss: %.2f acc: %.4f recall: %.4f F1: %.4f'
                % (ep_loss/(step+1), ep_acc/(step+1),ep_recall/(step+1),2*ep_acc/(step+1)*ep_recall/(step+1)/(ep_acc/(step+1)+ep_recall/(step+1))))
        


In [ ]:
batch_size = 2
output_size = 2
hidden_size = 256
embedding_length = 100
vocab_size=10000 # is not used
word_embeddings=None # is not used
self_atten_model=SelfAttention(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
self_atten_model=train_attn(self_atten_model)

using GPU


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


train step: 5900 loss: 0.12 acc: 0.9605 recall: 0.9604 F1: 0.9605
epoch: 0 loss: 0.12 acc: 0.9604 recall: 0.9603 F1: 0.9603
Validation:
using GPU
test step: 835 loss: 0.67 acc: 0.8594 recall: 0.8594 F1: 0.8594
test loss: 0.68 acc: 0.8590 recall: 0.8590 F1: 0.8590




In [ ]:
def make_prediction_val_attn(blstm):
    blstm.eval()
    with torch.no_grad():
        # test_data = torch.from_numpy(sent_val).to('cuda').float()
        predict=[]
        label=[]
        for step,(x,y) in enumerate(validateloader):
            x=x.float()
            y=y.float()
            if torch.cuda.is_available():
                x = x.to('cuda')
                y = y.to('cuda')
            output_predicted = blstm(x)
            output_predicted = F.sigmoid(output_predicted)
            accuracy_rat = 0.5
            pre_result = output_predicted  >= accuracy_rat
            pre_result = pre_result.float()
            # predict torch.cat([A, B], dim=0)
            predict+=(pre_result)
            label+=(y)
    return predict,label
predict,label=make_prediction_val_attn(self_atten_model)
pred_val=vector_to_label_lstm(predict)
label_val_int=vector_to_label_lstm(label)
print('\nLSTM val result:\n', classification_report(label_val_int, pred_val))


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")



LSTM val result:
               precision    recall  f1-score   support

           0       0.90      0.91      0.90      1457
           1       0.36      0.35      0.35       217

    accuracy                           0.83      1674
   macro avg       0.63      0.63      0.63      1674
weighted avg       0.83      0.83      0.83      1674



## output

In [ ]:
test_emb_data= trainset(senetance=test_emb,target=df_soltion.tweet_label_int)
test_emb_loader = DataLoader(test_emb_data, batch_size =2, shuffle = False, num_workers=1,pin_memory=True,drop_last=False)

In [ ]:
# predict the test data
def make_prediction_attn(blstm):
    blstm.eval()
    with torch.no_grad():
        # test_data = torch.from_numpy(sent_val).to('cuda').float()
        predict=[]
        label=[]
        for step,(x,y) in enumerate(test_emb_loader):
            x=x.float()
            # y=y.float()
            if torch.cuda.is_available():
                x = x.to('cuda')
                # y = y.to('cuda')
            output_predicted = blstm(x)
            output_predicted = F.sigmoid(output_predicted)
            accuracy_rat = 0.5
            pre_result = output_predicted  >= accuracy_rat
            pre_result = pre_result.float()
            # predict torch.cat([A, B], dim=0)
            predict+=(pre_result)
           
    return predict
# transfer vector to label
def vector_to_label_lstm(vec):
    output=[]
    
    for label in vec:
        flag=0
        if label[0]==1:
            flag=0
        else:
            flag=1
        output.append(flag)
    return output
#make kaggle file
def outputFile_attn(blstm):
    pre_result = make_prediction_attn(blstm)    
    pre_lsit=vector_to_label_lstm(pre_result)
    # create test file for kaggle
    df_soltion.tweet_label_int=pre_lsit
    df_soltion_out=df_soltion[['tweetid','tweet_label_int']]
    #And save it out to upload to competition!
    df_soltion_out.to_csv("my_kaggle_solution_lstm.csv",index=False)

In [ ]:
pre_result = make_prediction_attn(self_atten_model)    
pre_lsit=vector_to_label_lstm(pre_result)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
len(pre_lsit)

906

In [ ]:
outputFile_attn(self_atten_model)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# transformer



In [ ]:
!pip install simpletransformers

In [ ]:
from nltk import stem
import string
from gensim.parsing.preprocessing import remove_stopwords
Porter = stem.PorterStemmer()
import re
def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

def remove_user(vTEXT):
    # vTEXT = re.sub(r'(@)(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    vTEXT = re.sub(r'(@)(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

#token/stem/stopword_remove/ porcess
def data_pre_transfomer(sentence_list):
  sentences = []
  # this is a-z and space
  for sentence in sentence_list:
    if sentence is np.NaN:
          sentence=''
    # create a remove table for invalid character
    remove = remove_chars
    # There are some useless char in the sentence, so here I add and delete the data
    replace = " " * len(remove)
    # make a translate table
    remove_table = str.maketrans(remove , replace)
    #remove url
    senetence_removeURL=remove_urls(sentence)
    #REMOVE USER
    senetence_removeUrlAndUser=remove_user(senetence_removeURL)
    # clean punctuation and digits
    sentence_replace = senetence_removeUrlAndUser.lower().translate(remove_table)
    #clean stopwords and tokenize
    sentence_replace = remove_stopwords(sentence_replace)
    #remove meaningless tokens  also remove some HTML language
    sentence_new = []
    # for token in sentence_replace:
    #   # I think if a word are longer than 20, it have a great possiblity as a invalid word, and shorter than 2,it may be a stopword
    #   # so i only get the word that the length are form 2 to 20.
    #   if len(token) > 2 and len(token)<20:
    #     # lemmatizer, let the word become it's original shape, had --> have
    #     sentence_new.append(Porter.stem(token))
    sentences.append(sentence_replace)
  return sentences


In [ ]:
# used the data_pre method that defind above
train_data = data_pre_transfomer(df.text)
test_data = data_pre_transfomer(df_soltion.text)

# train_data = df.text
# test_data = df_soltion.text

In [ ]:
train_data[4]

'vaccinating hpv costs person avert million cervical cancer deaths time deliverforgood'

In [ ]:
print(df_soltion.text[13])
print(test_data[13])



Opinion | Pinterest takes the right step toward curbing misinformation on vaccines - The Washington Post https://t.co/8rp99QK3Jk
opinion pinterest takes right step curbing misinformation vaccines washington post


In [ ]:
from sklearn.utils import resample
df_df=pd.DataFrame({'text': list(np.array(train_data)),'lables': np.array(df.tweet_label_int.values) }, columns=[ 'text','lables'])

df_majority = df_df[df_df.lables==0]
df_minority = df_df[df_df.lables==1]


# 下采样多数类别
df_majority_downsampled = resample(df_majority,
                                 replace=False,    # sample without replacement
                                 n_samples=973,     # to match minority class
                                 random_state=123) # reproducible results
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

# Display new class counts
df_downsampled.lables.value_counts()



# df_minority_upsampled = resample(df_minority,
#                                  replace=True,     # sample with replacement
#                                  n_samples=7399,    # to match majority class
#                                  random_state=123) # reproducible results
# # 合并多数类别同上采样过的少数类别
# df_upsampled = pd.concat([df_majority, df_minority_upsampled])

# # 显示新的类别数量
# df_upsampled.lables.value_counts()

1    973
0    973
Name: lables, dtype: int64

In [ ]:
# !rm -rf /content/drive/My\ Drive/DL/best_model2

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data,df.tweet_label_int.values, test_size=0.1,
                                                    random_state=42) # so we get the same results  0.05

In [ ]:
y_train

array([0, 0, 0, ..., 0, 0, 1])

In [ ]:
df_df_up=pd.DataFrame({'text': list(np.array(X_train)),'lables': np.array(y_train)}, columns=[ 'text','lables'])

df_majority_up = df_df_up[df_df_up.lables==0]
df_minority_up = df_df_up[df_df_up.lables==1]
df_minority_upsampled = resample(df_minority_up,
                                 replace=True,     # sample with replacement
                                 n_samples=6663,    # to match majority class
                                 random_state=123) # reproducible results
# 合并多数类别同上采样过的少数类别
df_upsampled = pd.concat([df_majority_up, df_minority_upsampled])

# 显示新的类别数量
df_upsampled.lables.value_counts()



1    6663
0    6663
Name: lables, dtype: int64

In [ ]:
df_df_up_eval=pd.DataFrame({'text': list(np.array(X_test)),'lables': np.array(y_test)}, columns=[ 'text','lables'])

df_majority_up = df_df_up_eval[df_df_up_eval.lables==0]
df_minority_up = df_df_up_eval[df_df_up_eval.lables==1]
df_minority_upsampled = resample(df_minority_up,
                                 replace=True,     # sample with replacement
                                 n_samples=736,    # to match majority class
                                 random_state=123) # reproducible results
# 合并多数类别同上采样过的少数类别
df_upsampled_eval = pd.concat([df_majority_up, df_minority_upsampled])

# 显示新的类别数量
df_upsampled_eval.lables.value_counts()


1    736
0    736
Name: lables, dtype: int64

In [ ]:

from sklearn.utils import shuffle
df_upsampled = shuffle(df_upsampled)
df_upsampled_eval = shuffle(df_upsampled_eval)


df_upsampled_eval.lables.value_counts()

1    736
0    736
Name: lables, dtype: int64

In [ ]:


# eval_df = pd.DataFrame({'text': list(np.array(X_test)),'lables': np.array(y_test) }, columns=[ 'text','lables'])
# eval_df[-20:]
# eval_df.lables.value_counts()

In [ ]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [ ]:
!sh setup.sh

In [ ]:
# import torch
# torch.cuda.empty_cache()

In [ ]:
# reference:https://github.com/ThilinaRajapakse/simpletransformers#optional
# the rebert model is from simpletransformers org
from simpletransformers.classification import ClassificationModel
from simpletransformers.classification import ClassificationArgs
import pandas as pd
import logging
import sklearn

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. The first column is the text with type str, and the second column is the label with type int.
# train_data = [['Example sentence belonging to class 1', 1], ['Example sentence belonging to class 0', 0]]
# train_df = pd.DataFrame(X_train,y_train)

# eval_data = [['Example eval sentence belonging to class 1', 1], ['Example eval sentence belonging to class 0', 0]]
# eval_df = pd.DataFrame(eval_data)

# Create a ClassificationModel
model_args = ClassificationArgs()
# model_args.do_lower_case=True
model_args.overwrite_output_dir=True
model_args.output_dir='output_0802'
model_args.best_model_dir='output_0802'
model_args.evaluate_during_training=True
# model_args.evaluate_during_training_steps=3000
model_args.save_eval_checkpoints=False
model_args.warmup_steps=0
model_args.warmup_ratio=0.006
model_args.learning_rate=1e-6
model_args.num_train_epochs=20

# first time training 
# model_args.warmup_steps=0
# model_args.warmup_ratio=0.006
# model_args.learning_rate=1e-6
# model_args.num_train_epochs=20

# second time training 
# model_args.warmup_steps=0
# model_args.warmup_ratio=0.006
# model_args.learning_rate=1e-7
# model_args.num_train_epochs=20

model_args.train_batch_size=4
model_args.eval_batch_size=8

model_args.use_early_stopping = True
model_args.early_stopping_delta = 0.01  
model_args.early_stopping_metric = "mcc"  # disabled
model_args.early_stopping_metric_minimize = False
model_args.early_stopping_patience = 5
model_args.evaluate_during_training_steps = 1000
model_args.save_model_every_epoch=False
model_args.save_steps=0

model = ClassificationModel('roberta', '/content/drive/My Drive/DL/robert_0804', args=model_args) # You can set class weights by using the optional weight argument
#/content/drive/My Drive/DL/robert_0731/output_new
# Train the model
# model.train_model(df_upsampled,eval_df=df_upsampled_eval,mcc=sklearn.metrics.matthews_corrcoef)
# auc=sklearn.metrics.roc_auc_score
# Evaluate the model
#



In [ ]:
%cp -av  /content/output_0802/ /content/drive/My\ Drive/DL/robert_0804
# shutil.move("/content/outputs/best_model", "/content/out/best_model")

'/content/output_0802/' -> '/content/drive/My Drive/DL/robert_0804'
'/content/output_0802/checkpoint-3332-epoch-1' -> '/content/drive/My Drive/DL/robert_0804/checkpoint-3332-epoch-1'
'/content/output_0802/checkpoint-3332-epoch-1/eval_results.txt' -> '/content/drive/My Drive/DL/robert_0804/checkpoint-3332-epoch-1/eval_results.txt'
'/content/output_0802/checkpoint-6664-epoch-2' -> '/content/drive/My Drive/DL/robert_0804/checkpoint-6664-epoch-2'
'/content/output_0802/checkpoint-6664-epoch-2/eval_results.txt' -> '/content/drive/My Drive/DL/robert_0804/checkpoint-6664-epoch-2/eval_results.txt'
'/content/output_0802/eval_results.txt' -> '/content/drive/My Drive/DL/robert_0804/eval_results.txt'
'/content/output_0802/training_progress_scores.csv' -> '/content/drive/My Drive/DL/robert_0804/training_progress_scores.csv'
'/content/output_0802/config.json' -> '/content/drive/My Drive/DL/robert_0804/config.json'
'/content/output_0802/pytorch_model.bin' -> '/content/drive/My Drive/DL/robert_0804/pyt

In [ ]:
model.train_model(df_upsampled,eval_df=eval_df,args={'num_train_epochs': 5})

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

In [ ]:
predictions, raw_outputs = model.predict(list(np.array(X_test)))
predictions
print('\nBERT val result:\n', classification_report(np.array(y_test), predictions))




# BERT val result:
#                precision    recall  f1-score   support

#            0       0.91      0.95      0.93       362
#            1       0.56      0.40      0.47        57

#     accuracy                           0.88       419
#    macro avg       0.74      0.68      0.70       419
# weighted avg       0.86      0.88      0.87       419
# 74%


# BERT val result:
#                precision    recall  f1-score   support

#            0       0.93      0.90      0.91       362
#            1       0.46      0.54      0.50        57

#     accuracy                           0.85       419
#    macro avg       0.69      0.72      0.70       419
# weighted avg       0.86      0.85      0.86       419
# 64.5&



# BERT val result:
#                precision    recall  f1-score   support

#            0       0.91      0.93      0.92       736
#            1       0.44      0.36      0.40       102

#     accuracy                           0.87       838
#    macro avg       0.67      0.65      0.66       838
# weighted avg       0.86      0.87      0.86       838
# 66.666%


# BERT val result:
#                precision    recall  f1-score   support

#            0       0.91      0.94      0.92       736
#            1       0.43      0.34      0.38       102

#     accuracy                           0.86       838
#    macro avg       0.67      0.64      0.65       838
# weighted avg       0.85      0.86      0.86       838
# 69%


# BERT val result:
#                precision    recall  f1-score   support

#            0       0.92      0.91      0.92       736
#            1       0.42      0.45      0.44       102

#     accuracy                           0.86       838
#    macro avg       0.67      0.68      0.68       838
# weighted avg       0.86      0.86      0.86       838
# 80%

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.




BERT val result:
               precision    recall  f1-score   support

           0       0.92      0.91      0.92       736
           1       0.42      0.45      0.44       102

    accuracy                           0.86       838
   macro avg       0.67      0.68      0.68       838
weighted avg       0.86      0.86      0.86       838



In [ ]:
pre_lsit, raw_outputs =model.predict(list(np.array(test_data)))

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


In [ ]:
pre_lsit

In [ ]:
np.sum(pre_lsit)

111

In [ ]:
df_soltion.tweet_label_int=pre_lsit
df_soltion_out=df_soltion[['tweetid','tweet_label_int']]
#And save it out to upload to competition!
df_soltion_out.to_csv("/content/drive/My Drive/my_kaggle_solution_robert0804_up_leak.csv",index=False)